In [1]:
from qiskit import QuantumCircuit, Aer, execute
import json
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

# Generate the shared secret key using QKD with the BB84 protocol
def generate_key():
    # Initialize the quantum circuit
    qc = QuantumCircuit(16,64)
    
    # Create a random bit string
    key = ''
    for i in range(16):
        key += str(random.randint(0, 1))
    
    # Encode the key using BB84 protocol
    for i in range(len(key)):
        if key[i] == '0':
            qc.h(i)
        else:
            qc.x(i)
            qc.h(i)
    
    # Measure the qubits and extract the key
    qc.measure([0, 1], [0, 1])
    backend = Aer.get_backend('qasm_simulator')
    counts = execute(qc, backend, shots=1).result().get_counts()
    qkd_key = list(counts.keys())[0]
    
    # Return the QKD key
    return qkd_key




In [2]:
import PyPDF2
print(PyPDF2.__version__)

3.0.1


In [4]:
import random
import os
import PyPDF2

qkd_key=generate_key()

def encrypt_pdf(filename, qkd_key):
    # Open the PDF file and read the contents
    with open(filename, 'rb') as file:
        pdf_reader = PyPDF2.PdfFileReader(file)
        pdf_writer = PyPDF2.PdfFileWriter()
        for page_num in range(pdf_reader.getNumPages()):
            pdf_writer.addPage(pdf_reader.getPage(page_num))
        plaintext = pdf_writer.write_to_bytes()
    
    # Convert the QKD key to a byte string
    key = bytes.fromhex(qkd_key)
    
    # Generate a random initialization vector (IV) for AES
    iv = get_random_bytes(AES.block_size)
    
    # Encrypt the data using AES
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = iv + cipher.encrypt(pad(plaintext, AES.block_size))
    
    # Write the encrypted data to a new file
    with open(filename + '.enc', 'wb') as file:
        file.write(ciphertext)

encrypt_pdf('data.pdf', qkd_key)

def decrypt_pdf(filename, qkd_key):
    # Convert the QKD key to a byte string
    key = bytes.fromhex(qkd_key)
    
    # Open the encrypted file and read the contents
    with open(filename, 'rb') as file:
        ciphertext = file.read()
    
    # Extract the initialization vector (IV) from the ciphertext
    iv = ciphertext[:AES.block_size]
    
    # Decrypt the data using AES
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext[AES.block_size:])
    
    # Remove padding and write the decrypted data to a new file
    with open('dec_' + filename[:-4] + '.pdf', 'wb') as file:
        file.write(plaintext)

decrypt_pdf('dec_data.pdf', qkd_key)


NameError: name 'PdfFileReader' is not defined